In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
os.chdir("/home/manobhav/PycharmProjects/FirCrawler/Dataset/")

In [3]:
def savefile(dataframe,name):
    if os.path.isfile(name):
        with open(name, 'a') as f:
            dataframe.to_csv(f, header=False,index=False)
    else:
        dataframe.to_csv(name, index=False)

In [4]:
df1=pd.read_csv('new_raw_data.csv')
if os.path.isfile('raw_data.csv'):
    df2=pd.read_csv('raw_data.csv')
    df2=df2.append(df1, ignore_index=True)
    df2.drop_duplicates(subset="filename",inplace=True)
    df2.to_csv('raw_data.csv',index=False)
else:
    df1.to_csv('raw_data.csv',index=False)

In [5]:
import re
def cleaner(x):
    #print(x.psdate)
    arr=['dateto','datefrom','psdate','timeto','timefrom','pstime','section','pahar']
    for i in arr:
        x[i]=x[i].replace(" ","").replace("l","/").replace("hrs","").replace("N","A/").replace("Pahar","").replace("Z","2").replace("!","/")
            
    
    x.address=x.address.replace("|","I")
    x.day=x.day.replace("Fﬁday","Friday").replace("D10","DAY")
            
        
    return x

df1=df1.replace(np.nan, '', regex=True)
df1=df1.apply(cleaner,axis=1)
df1["pahar"] = np.where(df1.pahar.isnull(),0, df1.pahar)

In [6]:
df1.psdate=pd.to_datetime(df1.psdate,dayfirst=True,errors='coerce')
df1.datefrom=pd.to_datetime(df1.datefrom,dayfirst=True,errors='coerce')
df1.dateto=pd.to_datetime(df1.dateto,dayfirst=True,errors='coerce')

In [7]:
savefile(df1,"cleaning_fulldataset.csv")

In [8]:
off_ref=pd.read_csv("offence_ref.csv")
off_ref=off_ref[~off_ref.Offence.isnull()]
off_ref.Section=off_ref.Section.astype('str')


In [9]:
serious_charges=["house-trespass","Theft","Snatching","Stalking","Rash driving","Robbery","Kidnapping","domestic abuse","culpable homicide","sexual harrasement","Rape","murder"]
def func(x):
    charges=set()
    x["full_charge"]=""
    x["group_charge"]=""
    sections=x.section.split("/")
    for section in sections:
        section=section.replace("(","").replace(")","").replace("O","0")
        if len(off_ref[off_ref.Section==section])>0:
            charges.add(off_ref[off_ref.Section==section].loc[:,"Offence"].values[0])

    
    
    sorted_charges=sorted(charges)
    #print(sorted_charges)
    res=("-").join(sorted_charges)
    x["full_charge"]=res
    for charge in serious_charges:
        if charge in sorted_charges:
            res=charge
    x["group_charge"]=res
    #print(res)
    return x
df1.section=df1.section.astype('str')            
df1=df1.apply(func,axis=1)

In [10]:
counts=df1.group_charge.value_counts()
pd.set_option('display.max_rows', 500)
counts

Theft                                                                           1041
Disobedience to pub. Serv. Order                                                 678
Snatching                                                                        537
                                                                                 465
Rash driving                                                                     464
Kidnapping                                                                       332
obstruction by a person                                                          230
cheating                                                                         202
sexual harrasement                                                               192
Robbery                                                                          179
domestic abuse                                                                   132
Rape                                                             

In [11]:
extras=df1[(df1.full_charge=="")]
df1=df1[~(df1.full_charge=="")]

In [12]:
def func2(x):
    #print(len(x))
    if len(x)<3:
        x.group_charge="Others"
    return x
df1=df1.groupby(["group_charge"]).apply(func2)
df1.group_charge.value_counts()

Theft                                                      1041
Disobedience to pub. Serv. Order                            678
Snatching                                                   537
Rash driving                                                464
Kidnapping                                                  332
obstruction by a person                                     230
cheating                                                    202
sexual harrasement                                          192
Robbery                                                     179
domestic abuse                                              132
Rape                                                         78
Causing hurt-wrong restraint                                 75
obstruction by a person-public nuisance                      62
Others                                                       58
Criminal breach of trust-cheating                            45
Causing hurt-Criminal intimidation-wrong

In [13]:
df1["hour"]=pd.to_datetime(df1.timefrom).dt.hour
df1=df1[~df1.hour.isnull()]

In [14]:
savefile(df1,"charged_main_dataset_2.csv")

In [17]:
wrong_add=df1[(df1.address.str.contains("Distance")).all() or (df1.address.str.contains(re.compile(r'[a-z]+')))]
savefile(wrong_add,'wrong_add.csv')

In [18]:
add_clean=df1[~(df1.address.str.contains("Distance")).all() and ~(df1.address.str.contains(re.compile(r'[a-z]+')))]
savefile(add_clean,'cleaned_address_data_with_charges_3.csv')